In [6]:
import ee
import geemap
import numpy as np

try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [17]:
#draw a rectangle in the above map and then run this
feature = Map.draw_last_feature
roi = feature.geometry()
roi.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-120.021748, 46.126847],
   [-120.021748, 46.126957],
   [-120.021535, 46.126957],
   [-120.021535, 46.126847],
   [-120.021748, 46.126847]]]}

In [47]:
#1 Acre = 4047 m²
#1 Acre  = 0.404686 hectare

meterToAcre = (1/4047)
acreToHectare = 0.404686

In [48]:
# To check total area  of selected polygon 
coordinate = roi.getInfo()['coordinates']

polygon = ee.Geometry.Polygon(coordinate)
area = polygon.area().getInfo()
acre = area * meterToAcre
print('area: ', area,"m²")
print('Acre: ', acre)
print('Hectare: ', acre * acreToHectare)

area:  200.77803438014993 m²
Acre:  0.0496115726167902
Hectare:  0.02007710887599836


In [49]:
#Calculate area of 1 pixel

"""
Limitation:

It works only for recatange or square geometry

As we are creating a numpy matrix from the satellite image.
Number of columns should be same for all the rows. 

To do (if have time):
1. Create a new two-dimensional matrix representing mask of original geomentry
having values of 1 and 0

2. numpy matrix = numpy matrix * new matrix mask

"""

def maskS2clouds(image):
  qa = image.select('QA60')

  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
  return image.updateMask(mask).divide(10000) 


# Load Sentinel-2 TOA reflectance data.
imageCollection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2016-01-01', '2016-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .map(maskS2clouds) \
    .filterBounds(roi) \
    .sort('CLOUDY_PIXEL_PERCENTAGE')


count = imageCollection.size().getInfo()
print("Total satellite images available across the date range:",count)


rawImg = ee.Image(imageCollection.toList(imageCollection.size()).get(0))

"""
#To calculate pixel from actual image
pixelCountImage = rawImg.select(['B2'].reduceRegion(
  reducer = ee.Reducer.count(),
  geometry = roi
  )

TotalPixel = pixelCountImage.getInfo()['B2']
print("Total Pixel:",TotalPixel)

"""
matrix = geemap.ee_to_numpy(rawImg.select(['B2']), region=roi)

TotalPixel = matrix.shape[0]*matrix.shape[1]

print("Total Pixel:",TotalPixel)

areaOfOnePixel = area/TotalPixel
print("area of one pixel:",areaOfOnePixel,"acre")

Total satellite images available across the date range: 28
Total Pixel: 4
area of one pixel: 50.19450859503748


In [61]:
#sample predicted matrix from the model
samplePrediction = np.array([
    [0,1],
    [1,2]
])


#each value in the samplePrediction represent crop ID. 
crops = {
    0:"No crop",
    1:"Maize",
    2:"Corn"
}

type(samplePrediction)


numpy.ndarray

In [57]:
predictionCount = np.unique(samplePrediction, return_counts=True)
crop,totalcount = predictionCount[0],predictionCount[1]
crop,totalcount

(array([0, 1, 2]), array([1, 2, 1]))

In [63]:
def calculateArea(cropCount):
    return cropCount*meterToAcre*areaOfOnePixel


for cropId,cropCount in zip(crop,totalcount):
    print("{} : {} acre".format(crops[cropId],calculateArea(cropCount)))

No crop : 0.01240289315419755 acre
Maize : 0.0248057863083951 acre
Corn : 0.01240289315419755 acre
